## Data Wrangling Start/End Points of Colorado Oil and Gas Flowlines for Mapping

This Python notebook contains the methodology and code that Inside Energy used to clean the flowline data released by the Colorado Oil and Gas Conservation Commission June 27, 2017. This is the first time that data on the location of oil and gas flowlines is publicly available. You can view the completed map and read the accompanying web article at [InsideEnergy.org]()

The data contains the start and end locations of different types of flowlines: process piping, wellsite flowlines, dump lines, etc. It does not include the actual routes of the flowlines. We played around with a few ways to visualize this data on a map, including just showing the point locations of the start and end of each line. Ultimately, we decided to show a line connecting the start and end points. Note that this does not indicate the exact flowline route. However, we felt that this was the most useful way to quickly and visually identify where the flowlines are.

This notebook contains the code for the data cleaning process. Additional mapping was then done using QGIS and Carto. Final CSV files and additional notes are available in our [GitHub repository](..).

Work by Jordan Wirfs-Brock for Inside Energy. Questions or comments? Contact jordanwb@gmail.com.


In [1]:
import pandas as pd

#### Download and import the data
We downloaded the [Flowline Inventory](https://cogcc.state.co.us/documents/data/downloads/Engineering/flowline/FlowlineDownload.html) data from the COGCC website as a zipped file. The unzipped file contains an Excel file and Access database (.mdb) as well as a data dictionary, which you can refer to for a full list of the fields contained in the database. Going forward, COGCC will update the data weekly.

There's a *lot* of information in the flowline file, but we'll be focusing on the ID, the start and end latitude/longitude pairs, the operator and the flowline type.

In [2]:
# Import original data from Excel file
original = pd.read_excel("Flowline/FlowlineInventory.xlsx", index=False)

#### Format the data so that we can draw lines between start/end points

Eventually, we will be importing this data into QGIS and using the [Points2One](https://plugins.qgis.org/plugins/points2one/) plugin. To do this, our data will need to be formatted like this:

id | order | x | y
--- | --- | --- | ---
id1 | 1 |     x_start | y_start
id1 | 2 |     x_end |   y_end
id2 | 1 |     x_start | y_start
id2 | 2 |     x_end |   y_end

The ID column is the unique identifier provided by COGCC. In the order column, 1 is the start point of the flowline and 2 is the endpoint. (Friendly reminder: x is the longitude and y is the latitude.)

In [3]:
# Create a start dataframe with starting lat lon coordinates
start = original[['ID', 'Start Riser Latitude', 'Start Riser Longitude']].copy()

# Add order value of 1
start["order"] = 1

# Create an end dataframe with ending lat lon coordinates
end = original[['ID', 'End Point Latitude', 'End PointLongitude']].copy()

# Add order value of 2
end["order"] = 2

# Rename column names - we want them to be identical so that we can merge them
end.columns = ['ID', 'Latitude','Longitude','order']
start.columns = ['ID', 'Latitude','Longitude','order']

We now have two dataframes, one for the start points and one for the end points. Here's what they look like:

In [4]:
start

,ID,Latitude,Longitude,order
0,2,40.955840,-102.542530,1
1,4,40.955840,-102.542530,1
2,5,40.955850,-102.539210,1
3,7,40.955850,-102.539210,1
4,13,40.960750,-102.539800,1
5,14,40.960750,-102.539800,1
6,19,38.527507,-102.440313,1
7,23,38.885761,-102.119920,1
8,25,38.893710,-102.344810,1
9,26,38.895233,-102.259758,1


In [5]:
end

,ID,Latitude,Longitude,order
0,2,40.955870,-102.546190,2
1,4,40.955840,-102.542530,2
2,5,40.939450,-102.581140,2
3,7,40.955880,-102.539290,2
4,13,40.960980,-102.539810,2
5,14,40.960750,-102.539810,2
6,19,38.616964,-102.437339,2
7,23,38.863780,-102.089189,2
8,25,38.898788,-102.345201,2
9,26,38.903559,-102.246085,2


But what we really want is one data frame with the start and end points. So let's concatenate!

In [6]:
# Append end to start to create a dataframe that has both values
start_end = pd.concat([start,end])

Now let's see what it looks like:

In [7]:
start_end

,ID,Latitude,Longitude,order
0,2,40.955840,-102.542530,1
1,4,40.955840,-102.542530,1
2,5,40.955850,-102.539210,1
3,7,40.955850,-102.539210,1
4,13,40.960750,-102.539800,1
5,14,40.960750,-102.539800,1
6,19,38.527507,-102.440313,1
7,23,38.885761,-102.119920,1
8,25,38.893710,-102.344810,1
9,26,38.895233,-102.259758,1


In [8]:
# To get an idea for what the data looks like, let's do a describe
start_end.describe()

,ID,Latitude,Longitude,order
count,257652.000000,2.121770e+05,212176.000000,257652.000000
mean,64413.500000,3.012368e+03,-105.155232,1.500000
std,37188.935057,2.064047e+05,4.004101,0.500001
min,1.000000,-1.081237e+02,-1104.837335,1.000000
25%,32207.000000,4.003252e+01,-104.959770,1.000000
50%,64413.500000,4.016857e+01,-104.826980,1.500000
75%,96620.000000,4.030610e+01,-104.689660,2.000000
max,128826.000000,1.433487e+07,0.000000,2.000000


If you look at the data, you'll notice that there are several values that don't make sense. Some lat/lon points are missing completely, others are nonsensical (i.e., would put the end points of the flowlines in the middle of the South Pacific). So let's clean up the data a bit by removing the null values, fixing some data points where the lat/lons have been flipped, and dropping the values that are outside of Colorado.

How did we know to do this? Well, the first time we mapped it we got some strange results. Here are some screen grabs of what that pre-cleaned map looks like (interactive version [here](https://jordanwb.carto.com/viz/41275916-b96d-4762-9efd-89f2fbb34010/map)):

![Zoomed in view of weird flowlines](weird-flowlines-zoomed-in.png "Zoomed in view of weird flowlines")

![Zoomed out view of weird flowlines](weird-flowlines-zoomed-out.png "Zoomed out view of weird flowlines")

_We're pretty sure there isn't actually a flowline going from Colorado to Spain. So let's fix that._

In [9]:
# Drop null values
start_end.dropna(how='any', inplace="True")

# Test to see if any null values remain
start_end.isnull().sum()

ID           0
Latitude     0
Longitude    0
order        0
dtype: int64

In [10]:
start_end.describe()

,ID,Latitude,Longitude,order
count,212161.000000,2.121610e+05,212161.000000,212161.000000
mean,57625.901513,3.012592e+03,-105.155307,1.592017
std,36730.908883,2.064125e+05,4.004209,0.491461
min,1.000000,-1.081237e+02,-1104.837335,1.000000
25%,26532.000000,4.003252e+01,-104.959770,1.000000
50%,53425.000000,4.016857e+01,-104.826980,2.000000
75%,87305.000000,4.030611e+01,-104.689680,2.000000
max,128826.000000,1.433487e+07,0.000000,2.000000


In [11]:
# Drop values where latitude and longitude are 0
start_end = start_end[start_end.Latitude != 0]

In [12]:
# Check for values where longitude is < -110
start_end.query('Longitude <= -110')

,ID,Latitude,Longitude,order
95914,67994,39.951581,-140.603000,1
95915,67995,39.951581,-140.603000,1
128033,125342,37.014914,-214.936046,1
46000,48177,40.509043,-1104.837335,2
61678,68138,40.796520,-1014.846583,2
86550,47767,40.528664,-404.470748,2
93345,125009,38.993148,-170.861832,2


In [13]:
# Drop those values from the dataframe
start_end.drop(start_end[start_end.Longitude < -110].index, inplace=True)

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [14]:
# Check for values with Latitude > 42
start_end.query('Latitude > 42')
# And remove them
start_end.drop(start_end[start_end.Latitude > 42].index, inplace=True)



/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [15]:
# Check to see if it worked
start_end.query('Longitude <= -110')

,ID,Latitude,Longitude,order


In [16]:
# Check for values where the longitude is greater than -102
start_end.query('Longitude >= -102')

,ID,Latitude,Longitude,order
27772,51056,-108.051801,-39.366100,1
27773,51061,-108.056438,-39.378700,1
27774,51062,-108.056441,-39.378700,1
27775,51070,-108.056562,-39.367700,1
27776,51072,-108.058623,-39.375000,1
27777,51073,-108.061391,-39.376700,1
27778,51075,-108.061420,-39.376600,1
27779,51082,-108.063530,-39.374600,1
27780,51083,-108.063584,-39.374600,1
27781,51084,-108.063639,-39.374600,1


We can see that for these values, the latitude and longitude have been reversed and the longtidue value is negative when it should be positive. Let's fix it!

In [17]:
# First, create a new data frame from these values
to_flip = start_end.query('Longitude >= -102')

# And drop those values from the start_end dataframe
start_end.drop(start_end[start_end.Longitude >= -102].index, inplace=True)

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [18]:
# Now, let's flip the Longitude and Latitude columns by relabeling them
to_flip.columns= ['ID','Longitude','Latitude','order']

In [19]:
# Now we need to reverse the sign on Latitude
to_flip["Latitude"] = -to_flip["Latitude"]

/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [20]:
# Now concatenate with start_end dataframe to complete. Note: As we do this, we create a new data frame named start_end_cleaned.
start_end_cleaned = pd.concat([start_end, to_flip])

In [21]:
# Now do our checks again
start_end_cleaned.query('Longitude <= -110')

,ID,Latitude,Longitude,order


In [22]:
start_end_cleaned.query('Longitude >= -102')

# Drop remaining values - these ones were introduced from the switched lat/lon points
start_end_cleaned.drop(start_end_cleaned[start_end_cleaned.Longitude >= -102].index, inplace=True)

Now our data is cleaned and ready to go! Let's export it as a CSV, which we can then import into QGIS and use with the Points2One plugin.

In [23]:
# Export as CSV
start_end_cleaned.to_csv("All_Start_End.csv", header=True, index=False)

#### Create additional information lookup table
We'll also need a table that has the addition information -- operator and flowline type -- along with the IDs, so that we can join it to our flowlines shape files.

In [24]:
# Create flowline company and type table, to join to shape file once it's in Carto
flowline_info = original[['ID', 'Submitting Operator', 'Flowline Riser Type']].copy()
flowline_info.columns = ['ID', 'Operator','Type']

Before we finish, let's take a quick look at what we have by listing the unique values in the Operator and Type columns. We'll see that there are many different flowline types, and the labeling isn't consistent. That will be useful to know for later, when we make a map.

In [25]:
# Look for unique values in the Operator column
flowline_info.Operator.unique()

array(['AEON ENERGY CORP', 'MULL DRILLING COMPANY INC',
       'GRAYHORSE OPERATING INC', 'WARD PETROLEUM CORPORATION',
       'COMPLETE ENERGY SERVICES INC', 'SWEVCO - SABW LLC',
       'PETRON DEVELOPMENT COMPANY', 'NIGHTHAWK PRODUCTION LLC',
       'CONFLUENCE DJ LLC', 'GOSNEY & SONS INC',
       'RENEGADE OIL & GAS COMPANY LLC', 'BLUE CHIP OIL INC',
       'VERDAD RESOURCES LLC', 'PETERSON ENERGY OPERATING INC',
       'SG INTERESTS I LTD', 'STRACHAN EXPLORATION INC',
       'DIAMOND OPERATING, INC.', 'ENERVEST OPERATING LLC',
       'PEAKVIEW OPERATING COMPANY, LLC',
       'SANDRIDGE EXPLORATION & PRODUCTION LLC',
       'WHITING OIL & GAS CORPORATION', 'D J SIMMONS INC',
       'BAYSWATER EXPLORATION AND PRODUCTION LLC',
       'VANGUARD OPERATING LLC', 'TOM FENNO PRODUCTION LLC',
       'KERR MCGEE OIL & GAS ONSHORE LP', 'MCCARTNEY ENGINEERING LLC',
       'BP AMERICA PRODUCTION COMPANY', 'BLACK RAVEN ENERGY INC',
       'CONQUEST SEISMIC SERVICES INC', 'PROSPECT ENERGY LLC',
 

In [26]:
# Look for unique values in the Type column
flowline_info.Type.unique()

array(['meter', 'wellhead riser', 'compressor', 'separator',
       'separator/meter', nan, 'Dump Line', 'Well Site Flowline',
       'Process Piping', 'Sales Line', 'Non-Well Site Flowline',
       'Wellsite Flowline', 'Well Flowline Gas Outlet',
       'Well Flowline Produced Water Outlet',
       'Produced Water Transfer Station Dishcharge',
       'Gas Gathering Line Valve Set', 'Amherst Facility Discharge Line',
       'Process piping', 'Well Flowline Outlet', 'process piping',
       'PROCESS PIPING', 'Dump line', 'Well site flowline',
       'Well site Flowline', 'Non Well Site Flowline', 'Dump Lines',
       'STUB COMPRESSOR TO SKID', 'STUB FUEL GAS', 'STUB FROM COMPRESSOR',
       'STUB SALES', 'STUB FLASH GAS', 'STUB AT COMPRESSOR',
       'Fuel Gas System', 'Separator Bypass Line', 'Condensate',
       'HP Compressor Dump', 'Fuel Gas Line', 'Oil Dump line',
       'H20 Dump line', '1 inch supply line', 'Header', 'Line for 2 phase',
       'well site flowline', 'process Pipin

In [29]:
flowline_info["Operator"].value_counts()

KERR MCGEE OIL & GAS ONSHORE LP                       64920
PDC ENERGY INC                                        13295
NOBLE ENERGY INC                                      10375
VANGUARD OPERATING LLC                                 5923
CRESTONE PEAK RESOURCES OPERATING LLC                  5164
PIONEER NATURAL RESOURCES USA INC                      4164
TEP ROCKY MOUNTAIN LLC                                 2713
K P KAUFFMAN COMPANY INC                               1953
ENCANA OIL & GAS (USA) INC                             1850
GREAT WESTERN OPERATING COMPANY LLC                    1767
BAYSWATER EXPLORATION AND PRODUCTION LLC               1667
EXTRACTION OIL & GAS INC                               1159
SYNERGY RESOURCES CORPORATION                          1155
URSA OPERATING COMPANY LLC                             1147
LARAMIE ENERGY LLC                                      900
BP AMERICA PRODUCTION COMPANY                           864
BONANZA CREEK ENERGY OPERATING COMPANY L

Now we are ready to export the info file as a CSV. We can then import it into QGIS and join it with our flowlines shape files.

In [27]:
# Export as CSV
flowline_info.to_csv("All_Flowline_Info.csv", header=True, index=False)

Finishing up...

#### Draw lines between start/end points

In QGIS, we will:
1. Import the All_Start_End.csv file as text data
2. Map the points using the lat/long coordinates
3. Use the Points2One plugin to create a new shape file that draws lines between the start/end points of each flowline
4. Import the All_Flowline_Info.csv file as text data
5. Join that data with the flowlines, using the ID as a unique identifier
6. Calculate the length of each flowline, in meters
7. Save the new shape file (that has the polylines and the operator and type info) and export it

#### Create a map in Carto

In Carto, we will:
1. Import the flowlines shape file
2. Create a map
3. Color the lines based on flowline type
4. Filter out flowlines that are longer than 1500m
4. Add info windows

Here's the final product: LINK-TK